In [43]:
import pandas as pd 
import numpy as np 
from pathlib import Path 

In [44]:
ROOT = Path("..").resolve()

MSD = ROOT / "data" / "raw" / "msd"

SPOTATT = ROOT / "data" / "raw" / "spotatt"

music_info_path = MSD / "Music Info.csv"
listening_path = MSD / "User Listening History.csv"
spotatt_path = SPOTATT / "dataset.csv"

print("exists:", music_info_path.exists(), listening_path.exists(), spotatt_path.exists())


exists: True True True


In [45]:
music = pd.read_csv(music_info_path)
listen = pd.read_csv(listening_path)
spotifyAttr = pd.read_csv(spotatt_path)

print("music:", music.shape)
print("listen:", listen.shape)
print("spot:", spotifyAttr.shape)

music.head(2)

music: (50683, 21)
listen: (9711301, 3)
spot: (114000, 21)


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.0,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4


In [46]:
def normalize_spotify_id(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    s = s.str.replace("spotify:track:","", regex=False)
    s = s.str.extract(r"([A-za-z0-9]{22})", expand=False).fillna(s)
    return s


In [47]:
music["spotify_id_norm"] = normalize_spotify_id(music["spotify_id"])
spotifyAttr["spotify_id_norm"] = normalize_spotify_id(spotifyAttr["track_id"])

# quick sanity check: should mostly be length 22
print(music["spotify_id_norm"])
print(spotifyAttr["spotify_id_norm"])

0        09ZQ5TmUG8TSL56n0knqrj
1        06UfBBDISthj1ZJAtX4xjj
2        0keNu0t0tqsWtExGM3nT1D
3        0ancVQ9wEcHVd0RrGICTE4
4        01QoK9DA7VTeTSE3MNzp4I
                  ...          
50678    0tt1RdeJX1RyuU4hMEZ19T
50679    3wkdfXGf5JYErW4b35zP2h
50680    1gXMORZRGA40PE9rDE9cja
50681    1dxMDGvIYHFYgRvmw1uMHG
50682    3LW19oCCXxiqRcnlcQqPr3
Name: spotify_id_norm, Length: 50683, dtype: object
0         5SuOikwiRyPMVoIQDJUgSV
1         4qPNDBW1i3p13qLCt0Ki3A
2         1iJBSr7s7jYXzM8EGcbK5b
3         6lfxq3CG4xtTiEg7opyCyx
4         5vjLSffimiIP26QG5WcN2K
                   ...          
113995    2C3TZjDRiAzdyViavDJ217
113996    1hIz5L4IB9hN3WRYPOCGPw
113997    6x8ZfSoqDjuNa5SVP5QjvX
113998    2e6sXL2bYv4bSz6VTdnfLs
113999    2hETkH7cOfqmz3LqZDHZf5
Name: spotify_id_norm, Length: 114000, dtype: object


In [48]:
def unduplify_best(df, key):
    df = df.copy()
    df["filled_fields"] = df.notna().sum(axis=1)
    df = df.sort_values("filled_fields", ascending=False)
    df = df.drop_duplicates(subset=[key]).drop(columns=["filled_fields"])
    return df

In [49]:
spot_unique = unduplify_best(spotifyAttr, "spotify_id_norm")
spot_unique

,index,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,spotify_id_norm
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,5SuOikwiRyPMVoIQDJUgSV
75997,75997,02ZlHF9zknz6qKk2K8wqM1,Clannad,Macalla,The Wild Cry - Remastered 2003,20,280666,False,0.264,0.719,...,0,0.0551,0.2610,0.001080,0.0929,0.1550,87.073,4,new-age,02ZlHF9zknz6qKk2K8wqM1
76008,76008,212b6GfIExM6QKXA16NVE8,Josh Groban;Brian McKnight;London Symphony Orc...,Merry Christmas,Angels We Have Heard On High,1,210933,False,0.456,0.281,...,1,0.0302,0.9140,0.000586,0.1150,0.2590,110.009,4,opera,212b6GfIExM6QKXA16NVE8
76007,76007,220KFUVjdcPP0Nw0pUpyeU,Giacomo Puccini;Arioso Quartett Wien,Encores,Crisantemi,0,342893,False,0.150,0.106,...,0,0.0426,0.8420,0.835000,0.1110,0.0553,83.844,1,opera,220KFUVjdcPP0Nw0pUpyeU
76006,76006,2cEOeF7kmKInfVrk2A3yQU,Andrea Bocelli,Ya huele a buñuelos,Santa Claus llegò a la ciudad,0,214151,False,0.347,0.499,...,1,0.0344,0.7560,0.000000,0.1830,0.3450,133.872,4,opera,2cEOeF7kmKInfVrk2A3yQU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37992,37992,66FPjV3RMpyGvNpXgNgKRu,Mc Daleste;Studio Thg,As Melhores,Em teu olhar,42,176863,False,0.648,0.518,...,0,0.0355,0.7040,0.000000,0.1680,0.7610,124.994,4,funk,66FPjV3RMpyGvNpXgNgKRu
38008,38008,5TTGoX70AFrTvuEtqHK37S,Arctic Monkeys,AM,No. 1 Party Anthem,71,243131,False,0.505,0.698,...,1,0.0297,0.3300,0.088700,0.0966,0.5990,115.013,4,garage,5TTGoX70AFrTvuEtqHK37S
38007,38007,0SzvmWfOhoxZVGrmvb56YL,Arctic Monkeys,Leave Before The Lights Come On,Baby I'm Yours,69,152613,False,0.462,0.595,...,1,0.0347,0.2150,0.000000,0.1840,0.6400,113.719,4,garage,0SzvmWfOhoxZVGrmvb56YL
113999,113999,2hETkH7cOfqmz3LqZDHZf5,Cesária Evora,Miss Perfumado,Barbincor,22,241826,False,0.526,0.487,...,0,0.0725,0.6810,0.000000,0.0893,0.7080,79.198,4,world-music,2hETkH7cOfqmz3LqZDHZf5


In [50]:
merged = music.merge(spot_unique, on="spotify_id_norm", how="left", suffixes=("", "_spot"))
merged

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,loudness_spot,mode_spot,speechiness_spot,acousticness_spot,instrumentalness_spot,liveness_spot,valence_spot,tempo_spot,time_signature_spot,track_genre
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50678,TRQYCFV128F9322F50,Ryusei Rocket,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/d2668a5a3e0b1fda...,0tt1RdeJX1RyuU4hMEZ19T,NaN,NaN,2008,273440,0.438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50679,TRHQCSH128F42724B7,Colors Of The Wind,ACIDMAN,https://p.scdn.co/mp3-preview/8e22a7052ef3ecf7...,3wkdfXGf5JYErW4b35zP2h,"rock, alternative_rock, japanese, cover",NaN,2004,275133,0.351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50680,TRZRODK128F92D68D7,The Revelation,coldrain,https://p.scdn.co/mp3-preview/4b51a813f67e3853...,1gXMORZRGA40PE9rDE9cja,"metal, metalcore, post_hardcore",NaN,2014,254826,0.434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50681,TRGLMEM128F9322F63,Koi no Dependence,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/5a61e031df174666...,1dxMDGvIYHFYgRvmw1uMHG,NaN,NaN,2008,243293,0.513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
import numpy as np

def pick_genres(row):
    # priority: track_genre (often cleaner) -> genre -> tags
    if pd.notna(row.get("track_genre")):
        return [g.strip() for g in str(row["track_genre"]).split(",") if g.strip()]
    if pd.notna(row.get("genre")):
        return [g.strip() for g in str(row["genre"]).split(",") if g.strip()]
    if pd.notna(row.get("tags")):
        return [g.strip() for g in str(row["tags"]).split(",") if g.strip()]
    return []

merged["genres"] = merged.apply(pick_genres, axis=1)


In [52]:
merged

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,mode_spot,speechiness_spot,acousticness_spot,instrumentalness_spot,liveness_spot,valence_spot,tempo_spot,time_signature_spot,track_genre,genres
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[rock, alternative, indie, alternative_rock, i..."
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[rock, alternative, indie, pop, alternative_ro..."
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RnB]
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[rock, alternative, indie, alternative_rock, i..."
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RnB]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50678,TRQYCFV128F9322F50,Ryusei Rocket,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/d2668a5a3e0b1fda...,0tt1RdeJX1RyuU4hMEZ19T,NaN,NaN,2008,273440,0.438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
50679,TRHQCSH128F42724B7,Colors Of The Wind,ACIDMAN,https://p.scdn.co/mp3-preview/8e22a7052ef3ecf7...,3wkdfXGf5JYErW4b35zP2h,"rock, alternative_rock, japanese, cover",NaN,2004,275133,0.351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[rock, alternative_rock, japanese, cover]"
50680,TRZRODK128F92D68D7,The Revelation,coldrain,https://p.scdn.co/mp3-preview/4b51a813f67e3853...,1gXMORZRGA40PE9rDE9cja,"metal, metalcore, post_hardcore",NaN,2014,254826,0.434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[metal, metalcore, post_hardcore]"
50681,TRGLMEM128F9322F63,Koi no Dependence,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/5a61e031df174666...,1dxMDGvIYHFYgRvmw1uMHG,NaN,NaN,2008,243293,0.513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [53]:
merged.columns

Index(['track_id', 'name', 'artist', 'spotify_preview_url', 'spotify_id',
       'tags', 'genre', 'year', 'duration_ms', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'spotify_id_norm',
       'index', 'track_id_spot', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms_spot', 'explicit', 'danceability_spot',
       'energy_spot', 'key_spot', 'loudness_spot', 'mode_spot',
       'speechiness_spot', 'acousticness_spot', 'instrumentalness_spot',
       'liveness_spot', 'valence_spot', 'tempo_spot', 'time_signature_spot',
       'track_genre', 'genres'],
      dtype='object')

In [54]:

def coalesce_cols(df, base_col, spot_col, out_col):
    """
    out_col = base_col if not null else spot_col
    Works even if one of the columns is missing.
    """
    if base_col in df.columns and spot_col in df.columns:
        df[out_col] = df[base_col].combine_first(df[spot_col])
    elif base_col in df.columns:
        df[out_col] = df[base_col]
    elif spot_col in df.columns:
        df[out_col] = df[spot_col]
    else:
        df[out_col] = np.nan

# Audio features you have in BOTH forms (base + _spot)
coalesce_cols(merged, "tempo", "tempo_spot", "tempo_bpm_final")
coalesce_cols(merged, "energy", "energy_spot", "energy_final")
coalesce_cols(merged, "danceability", "danceability_spot", "danceability_final")
coalesce_cols(merged, "acousticness", "acousticness_spot", "acousticness_final")
coalesce_cols(merged, "valence", "valence_spot", "valence_final")
coalesce_cols(merged, "loudness", "loudness_spot", "loudness_final")
coalesce_cols(merged, "speechiness", "speechiness_spot", "speechiness_final")
coalesce_cols(merged, "instrumentalness", "instrumentalness_spot", "instrumentalness_final")
coalesce_cols(merged, "liveness", "liveness_spot", "liveness_final")
coalesce_cols(merged, "key", "key_spot", "key_final")
coalesce_cols(merged, "mode", "mode_spot", "mode_final")
coalesce_cols(merged, "time_signature", "time_signature_spot", "time_signature_final")

# Duration: you have duration_ms and duration_ms_spot
coalesce_cols(merged, "duration_ms", "duration_ms_spot", "duration_ms_final")

# Quick check
merged[[
    "tempo", "tempo_spot", "tempo_bpm_final",
    "energy", "energy_spot", "energy_final"
]].head(3)

C:\Users\josep\AppData\Local\Temp\ipykernel_23712\2435325962.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[out_col] = df[base_col].combine_first(df[spot_col])
C:\Users\josep\AppData\Local\Temp\ipykernel_23712\2435325962.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df[out_col] = df[base_col].combine_first(df[spot_col])
C:\Users\josep\AppData\Local\Temp\ipykernel_23712\2435325962.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determ

,tempo,tempo_spot,tempo_bpm_final,energy,energy_spot,energy_final
0,148.114,NaN,148.114,0.918,NaN,0.918
1,174.426,NaN,174.426,0.892,NaN,0.892
2,120.012,NaN,120.012,0.826,NaN,0.826


In [55]:
# Choose one genre field:
# You already have `genres` from the merged dataset. It likely contains tags/genre info.
# Keep it as a string for CSV. If it's already list-like, convert to string.
def ensure_string(x):
    if isinstance(x, list):
        return ", ".join([str(i) for i in x])
    return x

songs = pd.DataFrame({
    "msd_track_id": merged["track_id"],              # MSD ID (helps map listening history)
    "spotify_id": merged["spotify_id_norm"],         # canonical song id for indexing/ranking
    "title": merged["name"],
    "artist_name": merged["artist"],
    "preview_url": merged.get("spotify_preview_url"),
    "album_name": merged.get("album_name"),
    "release_year": merged.get("year"),

    # genre-ish fields (pick one)
    "genres": merged.get("genres").apply(ensure_string) if "genres" in merged.columns else merged.get("tags"),

    # popularity exists in spot dataset (no base popularity column shown)
    "popularity": merged.get("popularity"),

    # final audio features
    "tempo_bpm": merged.get("tempo_bpm_final"),
    "energy": merged.get("energy_final"),
    "danceability": merged.get("danceability_final"),
    "acousticness": merged.get("acousticness_final"),
    "valence": merged.get("valence_final"),
    "loudness": merged.get("loudness_final"),
    "speechiness": merged.get("speechiness_final"),
    "instrumentalness": merged.get("instrumentalness_final"),
    "liveness": merged.get("liveness_final"),
    "key": merged.get("key_final"),
    "mode": merged.get("mode_final"),
    "time_signature": merged.get("time_signature_final"),

    "duration_ms": merged.get("duration_ms_final"),

    # optional filter field from spot dataset
    "explicit": merged.get("explicit"),
})

# Clean up: drop rows without spotify_id, dedupe
songs = songs.dropna(subset=["spotify_id"]).drop_duplicates(subset=["spotify_id"])

songs.head(3), songs.shape


(         msd_track_id              spotify_id            title  artist_name  \
 0  TRIOREW128F424EAF0  09ZQ5TmUG8TSL56n0knqrj   Mr. Brightside  The Killers   
 1  TRRIVDJ128F429B0E8  06UfBBDISthj1ZJAtX4xjj       Wonderwall        Oasis   
 2  TROUVHL128F426C441  0keNu0t0tqsWtExGM3nT1D  Come as You Are      Nirvana   
 
                                          preview_url album_name  release_year  \
 0  https://p.scdn.co/mp3-preview/4d26180e6961fd46...        NaN          2004   
 1  https://p.scdn.co/mp3-preview/d012e536916c927b...        NaN          2006   
 2  https://p.scdn.co/mp3-preview/a1c11bb1cb231031...        NaN          1991   
 
                                               genres  popularity  tempo_bpm  \
 0  rock, alternative, indie, alternative_rock, in...         NaN    148.114   
 1  rock, alternative, indie, pop, alternative_roc...         NaN    174.426   
 2                                                RnB         NaN    120.012   
 
    ...  valence  loudness

In [56]:
keep_cols = [
    "spotify_id",
    "title",
    "artist_name",
    "genres",
    "duration_ms",
    "release_year",
    "tempo_bpm",
    "energy",
    "danceability",
    "acousticness",
    "valence",
]

songs_min = songs[keep_cols].copy()
songs_min.head(3), songs_min.shape


(               spotify_id            title  artist_name  \
 0  09ZQ5TmUG8TSL56n0knqrj   Mr. Brightside  The Killers   
 1  06UfBBDISthj1ZJAtX4xjj       Wonderwall        Oasis   
 2  0keNu0t0tqsWtExGM3nT1D  Come as You Are      Nirvana   
 
                                               genres  duration_ms  \
 0  rock, alternative, indie, alternative_rock, in...       222200   
 1  rock, alternative, indie, pop, alternative_roc...       258613   
 2                                                RnB       218920   
 
    release_year  tempo_bpm  energy  danceability  acousticness  valence  
 0          2004    148.114   0.918         0.355      0.001190    0.240  
 1          2006    174.426   0.892         0.409      0.000807    0.651  
 2          1991    120.012   0.826         0.508      0.000175    0.543  ,
 (50674, 11))

In [57]:
# fill missing genres so indexing won’t break
songs_min["genres"] = songs_min["genres"].fillna("unknown").astype(str)

# enforce numeric types
num_cols = ["duration_ms","release_year","tempo_bpm","energy","danceability","acousticness","valence"]
for c in num_cols:
    songs_min[c] = pd.to_numeric(songs_min[c], errors="coerce")

# drop rows missing required values (should be very few / none)
songs_min = songs_min.dropna(subset=num_cols)

# clamp 0-1 audio features
for c in ["energy","danceability","acousticness","valence"]:
    songs_min[c] = songs_min[c].clip(0, 1)

songs_min.isna().mean().sort_values(ascending=False).head(10)


spotify_id      0.0
title           0.0
artist_name     0.0
genres          0.0
duration_ms     0.0
release_year    0.0
tempo_bpm       0.0
energy          0.0
danceability    0.0
acousticness    0.0
dtype: float64

In [58]:
from pathlib import Path

out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "songs_min.csv"
songs_min.to_csv(out_path, index=False)

print("Saved:", out_path, "rows:", len(songs_min))


Saved: ..\data\processed\songs_min.csv rows: 50674
